In [ ]:
import os
import re

import nibabel as nib
import numpy as np

In [ ]:
det_path = "/home/yxy/disk/Repository/RibFrac/NoduleNet/results/cross_val_test/res/72"
gt_path = "/home/yutongx/src_data/labels"
save_path = "./"

gt_train_path = os.path.join(gt_path, "train")
gt_val_path = os.path.join(gt_path, "val")

save_train_path = os.path.join(save_path, "train")
save_val_path = os.path.join(save_path, "val")

In [ ]:
det_names = sorted(os.listdir(det_path))

if len(det_names) == 500:
    det_train_names = det_names[:420]
    det_val_names = det_names[-80:]
else:
    raise ValueError('detection names mismatch.')
    
gt_train_names = sorted(os.listdir(gt_train_path))
gt_val_names = sorted(os.listdir(gt_val_path))

In [ ]:
def masks2bboxes(masks, border):
    instance_nums = [num for num in np.unique(masks) if num]
    print('instance num:', instance_nums)
    bboxes = []
    
    for i in instance_nums:
        mask = (masks == i)
        if np.any(mask):
            zz, yy, xx = np.where(mask)
            bboxes.append([(zz.max() + zz.min()) / 2.,
                           (yy.max() + yy.min()) / 2.,
                           (xx.max() + xx.min()) / 2.,
                           zz.max() - zz.min() + 1 + border,
                           yy.max() - yy.min() + 1 + border,
                           xx.max() - xx.min() + 1 + border,
                           1])
    return bboxes

In [ ]:
# [prob, z_center, y_center, x_center, depth, height, width]

def foo(gt_names, det_names, save_path):
    for gt_name, det_name in zip(gt_names, det_names):
        print('gt_name:', gt_name)
        print('det_name:', det_name)
        
        gt_img = (np.swapaxis(nib.load(gt_name).get_fdata(), -1, 0)).astype(np.uint8)
        det_bbox = np.load(det_name, allow_pickle = True)##
        
        gt_bbox = masks2bboxes(gt_img, border=8)
        
        for gt in gt_bbox:
            det_sorted = sorted(det_bbox, key=lambda det: np.abs(det[1:4]-gt[:3]).sum())##
            
            for det in det_sorted:
                len_z = (det[4] + gt[3])/2 - abs(det[1] - gt[0])
                len_y = (det[5] + gt[4])/2 - abs(det[2] - gt[1])
                len_x = (det[6] + gt[5])/2 - abs(det[3] - gt[2])
                
                if len_z>0 and len_y>0 and len_x>0:### eliminate ambiguious cases
                    #det_pos
                    intsc = len_z*len_y*len_x
                    iou = intsc / ( det[4]*det[5]*det[6] + gt[3]*gt[4]*gt[5] - intsc 
                                   + np.finfo(intsc.dtype).eps) 